#### 1.1 Hdfs组件总览
1. Block
    * 数据块时hdfs文件系统存储的最小单元. 默认128M. 数据块越大
    * 读操作时: 客户端先到NameNode查看文件所包含的block块位置, 然后去到不同的DataNode读取数据  
      写操作是: 客户端首先到NameNode申请新的Block块, 用新Block块的位置信息建立数据流管道写数据

2. NameNode
    * NameNode主要维护2类信息:
        1. 文件系统的命名空间: 文件目录树, 文件/目录的相关信息
        2. 文件与Block块之间的索引关系
    * 以上信息使用2个文件记录在磁盘上:  
        1. fsimage: 命名空间的镜像文件
        2. editslog: 操作日志
    * 此外, NameNode还应该直到Block块与DataNode之间的对应关系, 这个关系没有记录在磁盘上, 而是在NameNode启动后, DataNode像NameNode动态汇报

3. DataNode
    * DataNode根据客户端或NameNode的请求将数据块读取或写入到本地
    * DataNode会不断向NameNode发送心跳, 汇报Block块以及缓存

#### 1.2 几个主要流程
1. 客户端读Hdfs的流程   
进行数据块读取操作的是 DFSInputStream
    1. 获取DataNode地址:(第三章)   
    客户端使用`DFSInputStream`的构造方法, 向NameNode询问该文件对应的第一个Block块在哪个DataNode上(`ClientProtocol.getBlockLocations()`), NameNode会返回一个距离客户端最近的DataNode的地址, 然后哭护短与其连接  
    2. 连接DataNode获取数据块(第四五章)     
    客户端调用`DFSInputStream.read()`方法, 和最优的DataNode建立连接, 数据会以数据包(packet)为单位流向客户端. 当到达这个数据块的末尾时, `DFSInputStream`会继续调用`ClientProtocol.getBlockLocations()`方法获取下一个数据块的位置信息  
    3. 关闭DFSInputStream   
    客户端最后会调用`HdfsDataInputStream.close()`关闭流

2. 客户端写Hdfs的流程   
    1. 在NameNode的文件树中创建新节点   
       当客户端要创建一个新文件时, 首先调用`DistributedFileStream.create()`在文件系统中创建一个新文件.该方法主要是调用`ClientProtocol.create()`通知NameNode, NameNode首先在文件目录树指定路径下添加一个新文件, 然后在editlog中记录本次操作. 此后流程完毕, 返回客户端一个`DFSOutputStream`对象   
    2. 询问对应的DataNode写文件   
       得到`DFSOutputStream`对象后, 客户端调用其write方法. 该方法首先会调用`ClientProtocal.addBlock()`向NameNode申请一个新的数据块, NameNode会返回这个数据块的位置信息   

    3. 和DataNode建立连接写入数据   
       客户端和DataNode建立连接后就可以开始写入文件了, 写入文件采用ack确认机制保证文件正确传输:  
        * 写入`DFSOutputStream`的数据会先缓存在数据流中, 并被切分成一个一个的数据包(packet)发往DataNode   
        * DataNode会对每个packet进行确认, 确认后的数据包会从缓存队列中删除  
        * 然后调用`ClientProtocal.addBlock()`向NameNode申请新的数据块   

     如果在数据传输过程中, DataNode挂掉了, 则`DFSOutputStream`进行如下操作来执行故障恢复  
         * 缓存中没有被DataNode确认的packet会重新加入发送队列, `DFSOutputStream`调用`ClientProtocol.updateBlockForPipeline()`为数据块申请一个新的时间戳, 然后用新的时间戳建立数据流管道 (这种机制使得DataNode的时间戳会过期, DataNode重启之后, 由于数据块的时间戳与NameNode中记录的不一致而被删除, 保证集群中所有数据块的正确性)  


#### 1.3 DataNode与NameNode的沟通
DataNode启动后与NameNode的沟通主要有3个方面: 握手, 注册, 快汇报和缓存汇报
* DataNode启动后先要和Namenode确认双方的版本号是否一致. 通过`DataNodeProtocal.versionRequest()`  
* DataNode向NameNode注册, 通过`DataNode.register()`  
* DataNode将自己的数据块上报到NameNode, NameNode利用这些信息在内存中重建数据块和DataNode的映射关系  




1. 客户端和服务端的通信
    1. 客户端短先将请求序列化
    2. 然后调用Client的call()方法发送请求到远程的服务端
    ```java
    Writable call(RPC.RpcKind rpcKind, 
                  Writable rpcRequest,
                  ConnectionId remoteId, 
                  int serviceClass,
                  AtomicBoolean fallbackToSimpleAuth, 
                  AlignmentContext alignmentContext)
    ```
    3. 的服务端在收到请求后, 调用Server.call()方法响应请求
2. 客户端存根Stub程序  
    1. 存根程序的第个任务是将请求序列化   
       Hadoop使用RpcEngine接口抽象不同种类的序列化方式, 其中有2个重要方法:
        1. `getProxy`: 
            * 客户端会调用该方法获取一个本地接口的代理对象, 然后在该对象上调用本地接口方法
            * 该方法采用动态代理: 客户端调用程序在
        2. `getServer`